# Phase 4 Time Series Project

## Group Members
- William Itotia
- Firdosa Mohammed
- Frank Oyugi
- Esther Terry Munene

# HOUSING YOU TOO LTD TOP 5 BEST ZIP CODES

## Problem Statement

Housing You Too Ltd, a consultancy and real estate firm, has been approached by a client eager to invest in properties with the goal of achieving the highest possible Return on Investment (ROI). The client is looking for our expert guidance to identify the top 5 zip codes that promise the best ROI. Recognizing the complexity and layered nature of this question, we are committed to providing a thorough and data-driven analysis. To achieve this, we will utilize the extensive Zillow dataset, which contains historical real estate data. By analyzing trends, property values, and other key metrics, we aim to deliver the most accurate and insightful recommendations to guide our client's investment decisions for maximum profitability.

## Objectives

1. Identify the 5 best zip codes that will bring our clients the highest ROI.

2. Build a time series regression model in order to predict the 5 most profitable zip codes.

3. Using the dataset and the model created investigate the growth rate of the real estate industry in relation to sales value of properties.

4. Identify the 5 worst zip codes that our client should avoid in order to avoiding incurring losses.

## Challenges

1. Data Ambiguity and Unpredictability: Since we are using real-world data, there is a possibility of ambiguity and unpredictability, which can lead to some inaccuracies in our analysis and predictions.

2. Changing Market Conditions: Real estate markets are influenced by various external factors such as economic shifts, policy changes, and unforeseen events like natural disasters or pandemics. These factors can alter market conditions rapidly, making historical data less reliable for future predictions.

## Solutions

1. We will implement robust data cleaning and preprocessing techniques to handle missing values, outliers, and inconsistencies. We will also use advanced statistical methods and machine learning algorithms that can manage noisy data and enhance prediction accuracy.
